In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# melihat perangkat GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
blok_size = 8
batch_size = 4

cuda


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [3]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_bacth(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - blok_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+blok_size] for i in ix])
    print(x)
    y = torch.stack([data[i+1:i+blok_size+1] for i in ix])
    print(y)
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_bacth('train')
print('inputs : ')
# print(x.shape)
print(x)
print('targets : ')
print(y)

tensor([105738, 140530,  38434, 136609])
tensor([[58,  1, 73, 68,  1, 66, 54, 64],
        [65, 59,  1, 68, 59,  1, 58, 54],
        [68, 71, 58,  1, 73, 61, 54, 67],
        [ 1, 73, 61, 58, 66,  1, 62, 67]])
tensor([[ 1, 73, 68,  1, 66, 54, 64, 58],
        [59,  1, 68, 59,  1, 58, 54, 56],
        [71, 58,  1, 73, 61, 54, 67,  1],
        [73, 61, 58, 66,  1, 62, 67,  1]])
inputs : 
tensor([[58,  1, 73, 68,  1, 66, 54, 64],
        [65, 59,  1, 68, 59,  1, 58, 54],
        [68, 71, 58,  1, 73, 61, 54, 67],
        [ 1, 73, 61, 58, 66,  1, 62, 67]], device='cuda:0')
targets : 
tensor([[ 1, 73, 68,  1, 66, 54, 64, 58],
        [59,  1, 68, 59,  1, 58, 54, 56],
        [71, 58,  1, 73, 61, 54, 67,  1],
        [73, 61, 58, 66,  1, 62, 67,  1]], device='cuda:0')


In [5]:


x = train_data[:blok_size]
y = train_data[1:blok_size+1]

for t in range(blok_size):
    context = x[:t+1]
    target = y[t]
    print('when input is',context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [6]:
# class BigramLanguageModel(nn.Module):
#     def __init__(self, vocab_size):
#         super().__init__()
#         self.token_embeddings_table = nn.Embedding(vocab_size, vocab_size)

#     def forward(self, index, targets):
#         logits = self.token_embeddings_table(index)
#         B,T,C = logits.shape
#         logits = logits.view(B*T,C)
#         targets = targets.view(B*T)
#         loss = F.cross_entropy(logits, targets)

#         return logits

In [12]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = target.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # fokus only on the last time step
            logits = logits[:, -1, :] # become (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append samples index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Uv:pTUP tZ﻿p2zyMMEdBYJ7W8oLt*;S5?xTo_X&vbuLgFhlZ[u'o2OX(:2L*2gfz?3vc[6I!Y.np52xkw
U[9([u)nAXT)gG&SbVKvKKTDkb﻿muo&
'6Uhv4lw﻿DYNi;RP20"Q"_:aFpX8620 ([s?084nhsptC'yXER?!5pzMT:Wln7Jk ,13UPN,XX1H(d'0]jAZIuQ:QPMbCK7Z]J?2d3Y?]
gNWvICVJ,NJ[HyU4d.q(oGYr*b9[IR?N[U WPX0 npQCZP VW1&4gBr8TwKvyKws9O3I5.MDZc7'?uxxJ9k'X*z﻿,j]u)GcjVZz,e0Wa;_)kdN5w96'juyW8R?;)dnw?gGzmav.;fEZ[J*20X,V,a_6odNWT&UmhluDendjwzp 
u)b﻿"Btb!KFlt["XhpzWWgI!01Tsk7"s&&-FeAw9"xhmT'&ZP]QltpNlOQvW7"uLBI1U-zQ:p5pw.yUb.'dGgKZk_,*'gQfN;N0IPpy)*T3y
